<h2>Titter sentiment analysis</h2>

- You can also multiply score by the number of likes a particular tweet has received
- Display the word cloud for each candidate
- Display the data on a time series graph
- Include an image of the polls graph

https://projects.fivethirtyeight.com/polls/pennsylvania/

In [35]:
import os
import nltk
import tweepy
from dotenv import load_dotenv
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from datetime import datetime

In [6]:
import ssl
def set_up_ssl():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

In [7]:
set_up_ssl()
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dennisfenchenko/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
load_dotenv()
TWITTER_BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')
TWITTER_API_KEY = os.getenv('TWITTER_API_KEY')
TWITTER_API_SECRET_KEY = os.getenv('TWITTER_API_SECRET_KEY')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

In [9]:
print("Authenticating to Twitter...")

client = tweepy.Client(bearer_token=TWITTER_BEARER_TOKEN)
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET_KEY)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)

Authenticating to Twitter...


In [10]:
# create Twitter API client
api = tweepy.API(auth, wait_on_rate_limit=True)

In [87]:
# TEST CODE

tweets

[Status(_api=<tweepy.api.API object at 0x1089248b0>, _json={'created_at': 'Wed Oct 05 17:39:10 +0000 2022', 'id': 1577715031621439494, 'id_str': '1577715031621439494', 'full_text': "John Fetterman doesn't live in New Jersey, won't vote in Turkey's elections, and never killed puppies.\n\nHis opponent can't say that.", 'truncated': False, 'display_text_range': [0, 132], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1640929196, 'id_str': '1640929196', 'name': 'BrooklynDad_Defiant!☮️', 'screen_name': 'mmpadellan', 'location': 'Brooklyn, NY', 'description': "Proud papa. Perpetually pissed.\n#BlueWave2022, #Feminist. \n#IStandWithUkraine🇺

In [82]:
# TEST CODE

tweet_status = tweets[1]
print(tweet_status)
# print("tweets ", len(tweets))

Status(_api=<tweepy.api.API object at 0x1089248b0>, _json={'created_at': 'Sun Oct 09 19:44:21 +0000 2022', 'id': 1579196087578943488, 'id_str': '1579196087578943488', 'full_text': '‘Dr Oz has never stopped reminding me that I had a stroke,’ @JohnFetterman says at top of a rally in Bucks County.  ‘I guarantee there’s at least one person here filing me hoping to catch me missing some words. What kind of a doctor roots for somebody who was sick, stay sick?’', 'truncated': False, 'display_text_range': [0, 277], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'JohnFetterman', 'name': 'John Fetterman', 'id': 3622368202, 'id_str': '3622368202', 'indices': [60, 74]}], 'urls': []}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': Non

In [51]:
# TEST CODE

json_tweet_str = json.dumps(tweet_status._json)

json_tweet = json.loads(json_tweet_str)

In [52]:
# TEST CODE
json_tweet

{'created_at': 'Sun Oct 09 19:44:21 +0000 2022',
 'id': 1579196087578943488,
 'id_str': '1579196087578943488',
 'full_text': '‘Dr Oz has never stopped reminding me that I had a stroke,’ @JohnFetterman says at top of a rally in Bucks County.  ‘I guarantee there’s at least one person here filing me hoping to catch me missing some words. What kind of a doctor roots for somebody who was sick, stay sick?’',
 'truncated': False,
 'display_text_range': [0, 277],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'JohnFetterman',
    'name': 'John Fetterman',
    'id': 3622368202,
    'id_str': '3622368202',
    'indices': [60, 74]}],
  'urls': []},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_

In [53]:
# TEST CODE
json_tweet["full_text"]

'‘Dr Oz has never stopped reminding me that I had a stroke,’ @JohnFetterman says at top of a rally in Bucks County.  ‘I guarantee there’s at least one person here filing me hoping to catch me missing some words. What kind of a doctor roots for somebody who was sick, stay sick?’'

In [11]:
# TEST CODE

print("Creating Vader sentiment analyzer object")

sentiment = SentimentIntensityAnalyzer()

Creating Vader sentiment analyzer object


In [68]:
# TEST CODE
ls = ["what", "terrible", "idea"]
sentiment.polarity_scores("What a terrible idea!")

{'neg': 0.531, 'neu': 0.469, 'pos': 0.0, 'compound': -0.5255}

In [70]:
# TEST CODE
sentiment.polarity_scores(ls)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [12]:
def get_tweets_for_user(username, start_date, end_date, num_tweets):
    search_query = username + " -filter:retweets"
    tweets = api.search_tweets(q = search_query, count = num_tweets,
                               tweet_mode="extended",
                           lang="en", result_type="popular", since_id =
                           start_date, until = end_date)
    return tweets

In [17]:
print("Fetching tweets for two candidates")

SINCE_DATE = "2022-10-06"
UNTIL_DATE = "2022-10-13"
NUM_TWEETS = 1000

democratic_candidate_query = "@JohnFetterman OR John Fetterman OR Fetterman"

republic_candidate_query = "@DrOz OR Dr. Oz OR Dr Oz OR Dr. Mehmet Oz OR Mehmet Oz"

democratic_tweets = get_tweets_for_user(democratic_candidate_query,
                                        SINCE_DATE, UNTIL_DATE, NUM_TWEETS)

republican_tweets = get_tweets_for_user(republic_candidate_query, SINCE_DATE,
                                        UNTIL_DATE, NUM_TWEETS)

Fetching tweets for two candidates


In [25]:
print("The numeber of tweets fetch for each candidate is: ")
print("Democratic candidate ", len(democratic_tweets))
print("Republican candidate ", len(republican_tweets))

The numeber of tweets fetch for each candidate is: 
Democratic candidate  14
Republican candidate  13


In [26]:
def convert_tweets_to_json(raw_tweets):
    json_tweets = []
    for tweet in raw_tweets:
        json_tweet_str = json.dumps(tweet._json)
        json_tweet = json.loads(json_tweet_str)
        json_tweets.append(json_tweet)
    return json_tweets

In [27]:
print("Converting democratic tweets to json")
democratic_json_tweets = convert_tweets_to_json(democratic_tweets)

print("Converting republican tweets to json")
republican_json_tweets = convert_tweets_to_json(republican_tweets)

Converting democratic tweets to json
Converting republican tweets to json


In [28]:
def write_json_tweets_to_file(json_tweets, TWEETS_DIR, FILE_NAME):
    tweets_file = open(TWEETS_DIR + FILE_NAME, "a")
    tweets_file_str = json.dumps(json_tweets)
    tweets_file.write(tweets_file_str)
    tweets_file.close()

In [29]:
print("Writing tweets to a file")

TWEETS_DIR = "tweets/"

DEMOCRATIC_TWEETS_FILE_NAME = "democratic_tweets.json"
REPUBLICAN_TWEETS_FILE_NAME = "republican_tweets.json"

write_json_tweets_to_file(democratic_json_tweets, TWEETS_DIR, DEMOCRATIC_TWEETS_FILE_NAME)

write_json_tweets_to_file(republican_json_tweets, TWEETS_DIR, REPUBLICAN_TWEETS_FILE_NAME)

Writing tweets to a file


In [30]:
def read_tweets_from_file(file_name):
    with open(TWEETS_DIR + file_name, "r") as tweets_file:
        tweets = json.load(tweets_file)
    return tweets

In [31]:
print("Retrieving the tweets for both candidates from file")

READ_TWEETS_FROM_FILE = False

if READ_TWEETS_FROM_FILE:
    democratic_json_tweets = read_tweets_from_file(DEMOCRATIC_TWEETS_FILE_NAME)
    republican_json_tweets = read_tweets_from_file(REPUBLICAN_TWEETS_FILE_NAME)

Retrieving the tweets for both candidates from file


In [32]:
print("Creating Vader sentiment analyzer object")

sentiment = SentimentIntensityAnalyzer()

Creating Vader sentiment analyzer object


In [34]:
democratic_json_tweets[0]

{'created_at': 'Wed Oct 12 13:47:20 +0000 2022',
 'id': 1580193404708786178,
 'id_str': '1580193404708786178',
 'full_text': "I have a confession to make.\n\nI used a teleprompter in this interview last night. The truth is I am not able to do my show without a teleprompter. (That's true of every TV news host who is discussing the way John Fetterman does interviews.) https://t.co/EAK1TLcwBm",
 'truncated': False,
 'display_text_range': [0, 240],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/EAK1TLcwBm',
    'expanded_url': 'https://twitter.com/TheLastWord/status/1580158903450742786',
    'display_url': 'twitter.com/TheLastWord/st…',
    'indices': [241, 264]}]},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id

In [36]:
# this function retruns a dictionary containing
def get_tweet_sentiment_score(tweets):
    sentiments = []
    for tweet in tweets:
        timestamp = tweet["created_at"]
        datetime_timestamp = datetime.strftime(datetime.strptime(timestamp,'%a %b %d '
                                                                   '%H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        tweet_body = tweet["full_text"]
        retweet_count = tweet["retweet_count"]
        tweet_sentiment = sentiment.polarity_scores(tweet_body)
        tweet_neg_score = tweet_sentiment['neg']
        tweet_neu_score = tweet_sentiment['neu']
        sentiments.append((datetime_timestamp, tweet_neu_score,
                           tweet_neg_score, retweet_count))
    return sentiments

In [39]:
print("Getting sentiment scores for each candidate")

democratic_sentiments = get_tweet_sentiment_score(democratic_json_tweets)

republican_sentiments = get_tweet_sentiment_score(republican_json_tweets)

Getting sentiment scores for each candidate


In [42]:
print("Analysis for democratic candidate yields:")
print(democratic_sentiments)

Analysis for democratic candidate yields:
[('2022-10-12 13:47:20', 0.896, 0.0, 7129), ('2022-10-12 15:38:47', 0.785, 0.215, 4054), ('2022-10-12 15:19:48', 1.0, 0.0, 2428), ('2022-10-12 23:25:05', 1.0, 0.0, 3738), ('2022-10-12 19:29:29', 0.67, 0.274, 813), ('2022-10-12 22:06:11', 0.94, 0.06, 743), ('2022-10-12 20:01:09', 0.784, 0.159, 1040), ('2022-10-12 23:47:18', 0.528, 0.386, 813), ('2022-10-12 18:01:11', 0.899, 0.0, 893), ('2022-10-12 18:25:47', 1.0, 0.0, 731), ('2022-10-12 19:31:29', 0.809, 0.042, 1903), ('2022-10-12 19:11:56', 0.848, 0.0, 847), ('2022-10-12 17:19:44', 0.693, 0.307, 777), ('2022-10-12 13:30:02', 0.858, 0.068, 1340)]


In [43]:
print("Analysis for republican candidate yields:")
print(republican_sentiments)

Analysis for republican candidate yields:
[('2022-10-12 17:48:45', 0.754, 0.075, 830), ('2022-10-12 23:03:00', 0.69, 0.19, 41), ('2022-10-12 20:05:36', 0.769, 0.147, 32), ('2022-10-12 02:44:47', 1.0, 0.0, 193), ('2022-10-11 23:08:26', 0.892, 0.073, 384), ('2022-10-12 14:32:57', 0.517, 0.0, 43), ('2022-10-12 20:00:01', 0.856, 0.0, 27), ('2022-10-12 21:49:39', 0.958, 0.042, 21), ('2022-10-12 13:55:46', 0.705, 0.048, 28), ('2022-10-12 02:11:34', 0.713, 0.287, 88), ('2022-10-07 17:00:01', 0.759, 0.184, 1658), ('2022-10-06 21:09:39', 0.417, 0.0, 616), ('2022-10-12 01:33:01', 0.911, 0.036, 41)]


In [ ]:
# plot sentiment in a timeseries graph, you've got timestamps in the json file